In [8]:
using DataFrames, CSV, Statistics, MLJ, Random

import Pkg; Pkg.add("MLJModels")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching:

    Fetching: [====================>                    

Fetching: [========================================>]  100.0 %     ]  86.3 %Fetching: [========================================>]  97.6 %

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
  [d491faf4] + MLJModels v0.11.2
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [2]:
df = DataFrame(CSV.File("processed.cleveland.csv"))
rename!(df, [:age, :sex, :cp, :trestbps, :chol, :fbs, :restecg, :thalach, :exang, :oldpeak, :slope, :ca, :thal, :target]);

In [3]:
#limpando os dados

#permitindo missing data no df
allowmissing!(df)

#substituindo todos os dados faltosos "?" por missing values
df .= ifelse.(df .=="?", missing, df)

#removendo as linhas com missing values
dropmissing!(df)

#transformando as duas colunas em float64
df.ca = parse.(Float64, df.ca)
df.thal = parse.(Float64, df.thal);


In [4]:
y, X = unpack(df, ==(:target), colname -> true);

train, test = partition(eachindex(y), 0.7, stratify=y);

In [13]:
models(matching(X, y))

3-element Array{NamedTuple{(:name, :package_name, :is_supervised, :docstring, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :is_pure_julia, :is_wrapper, :load_path, :package_license, :package_url, :package_uuid, :prediction_type, :supports_online, :supports_weights, :input_scitype, :target_scitype, :output_scitype),T} where T<:Tuple,1}:
 (name = EvoTreeCount, package_name = EvoTrees, ... )
 (name = LinearCountRegressor, package_name = GLM, ... )
 (name = XGBoostCount, package_name = XGBoost, ... )

In [10]:
using MLJ

DTCModel = @load 

tree = machine(DTCModel, X, y)

fit!(tree, rows=train);

┌ Warning: The scitype of `y`, in `machine(model, X, y, ...)` is incompatible with `model`:
│ scitype(y) = AbstractArray{Count,1}
│ target_scitype(model) = AbstractArray{#s31,1} where #s31<:Finite.
└ @ MLJBase /home/brainiac/.julia/packages/MLJBase/2yoMe/src/machines.jl:81
┌ Info: Training Machine{DecisionTreeClassifier} @170.
└ @ MLJBase /home/brainiac/.julia/packages/MLJBase/2yoMe/src/machines.jl:317


DomainError: DomainError with 2:
Can only convert categorical elements to integers. 